<a href="https://colab.research.google.com/github/ananya0-0/Hand-Gesture-Recognition-/blob/main/Hand_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!git config --global user.email "ananychauhan365@gmail.com"
!git config --global user.name "ananya365"

!rm -rf Hand-Gesture-Recognition- #deleting incorrect folder

!GIT_TERMINAL_PROMPT=0 git clone https://@github.com/ananya0-0/Hand-Gesture-Recognition.git

%cd Hand-Gesture-Recognition

In [ ]:
!pip install opencv-python mediapipe numpy

UPLOADING KAGGLE API TOKEN

In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d grassknoted/asl-alphabet

!unzip asl-alphabet.zip -d asl_alphabet_train

REQUIRED SETUP LIBRARIES ETC

In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
dataset_dir = '/content/asl_alphabet_train/asl_alphabet_train'

model_save_dir = '/content/ASL_Model'
model_path = os.path.join(model_save_dir, 'asl_classifier.pkl')
label_map_path = os.path.join(model_save_dir , 'label_map.pkl')

#global lists
features = []
labels = []
label_map = {} # will store the names
current_label_id = 0

#initializing hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)
print(f"Setup complete. Reading data from local path: {dataset_dir}")